In [ ]:
# !pip install -U transformers[torch] datasets[audio] accelerate nltk sacrebleu rouge-score py7zr evaluate bertviz huggingface_hub

In [ ]:

from transformers import pipeline, set_seed

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Summarization

## The CNN/DailyMail Dataset

In [ ]:
from datasets import load_dataset


dataset = load_dataset("cnn_dailymail", "3.0.0", trust_remote_code=True)

In [ ]:
sample = dataset["train"][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])

## Text Summarization Pipelines

In [ ]:
sample_text = dataset["train"][1]["article"][:2000]

summaries = {}

In [ ]:


import sys

if 'nltk' in sys.modules:
    del sys.modules['nltk']

import nltk

try:
    print(f"NLTK Version: {nltk.__version__}")
    nltk.download("punkt")
    nltk.download("punkt_tab")
    nltk.download("wordnet")
    from nltk.tokenize import sent_tokenize
    print(" NLTK พร้อมใช้งานแล้ว!")
except AttributeError:
    print(" ยังติดปัญหาเดิม ลองกด Runtime -> Restart session อีกครั้ง")

In [ ]:
string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

### Summarization Baseline


In [ ]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [ ]:
summaries["baseline"] = three_sentence_summary(sample_text)

### GPT-2

In [ ]:
#hide_output
from transformers import pipeline, set_seed

set_seed(42)
pipe = pipeline("text-generation", model="gpt2-xl")
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(
    sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))

In [ ]:
print("--- GROUND TRUTH (เฉลย) ---")
print(dataset["train"][0]["highlights"])

print("\n--- GPT-2 GENERATED SUMMARY ---")
print(summaries["gpt2"])

### T5

<img alt="T5" width="700" caption="Diagram of T5's text-to-text framework (courtesy of Colin Raffel); besides translation and summarization, the CoLA (linguistic acceptability) and STSB (semantic similarity) tasks are shown" src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter08_t5.png?raw=1" id="T5"/>

In [ ]:
#hide_output
pipe = pipeline("summarization", model="t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

### BART

In [ ]:
#hide_output
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

### PEGASUS

<img alt="pegasus" width="700" caption="Diagram of PEGASUS architecture (courtesy of Jingqing Zhang et al.)" src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter08_pegasus.png?raw=1" id="pegasus"/>

In [ ]:
#hide_output
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")
pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

In [ ]:
print("--- [GROUND TRUTH / เฉลย] ---")
print(dataset["train"][1]["highlights"])

print("\n--- [GPT-2 SUMMARY] ---")
print(summaries.get("gpt2", "ยังไม่ได้รัน หรือรันไม่สำเร็จ"))

print("\n--- [PEGASUS SUMMARY] ---")
print(summaries["pegasus"])

## Comparing Different Summaries

In [ ]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

## Measuring the Quality of Generated Text

### BLEU

In [ ]:
!pip install evaluate
# hide_output
import evaluate

# เปลี่ยนจาก load_metric เป็น evaluate.load
bleu_metric = evaluate.load("sacrebleu")

In [ ]:
import pandas as pd
import numpy as np

bleu_metric.add(
    prediction="the the the the the the", reference=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

In [ ]:
bleu_metric.add(
    prediction="the cat is on mat", reference=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

### ROUGE

In [ ]:
# hide_output
import evaluate

# เปลี่ยนจาก load_metric เป็น evaluate.load
# 'rouge' ในไลบรารีใหม่จะคำนวณ ROUGE-1, ROUGE-2, ROUGE-L ให้ครบในตัวเดียว
rouge_metric = evaluate.load("rouge")

In [ ]:
import pandas as pd

reference = dataset["train"][8]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    # 1. ใส่ข้อมูลเข้า Metric
    rouge_metric.add(prediction=summaries[model_name], reference=reference)

    # 2. คำนวณคะแนน (เวอร์ชันใหม่คืนค่าเป็น dict ของ float เลย)
    score = rouge_metric.compute()

    # 3. ดึงค่าคะแนนออกมา (คูณ 100 เพื่อให้เป็นเปอร์เซ็นต์เหมือนในหนังสือ)
    rouge_dict = {rn: score[rn] * 100 for rn in rouge_names}
    records.append(rouge_dict)

# แสดงผลเป็น DataFrame
pd.DataFrame.from_records(records, index=summaries.keys())

In [ ]:
# ลองส่งข่าว 1 ชิ้นเข้าไปให้มันสรุปเลย
sample_text = dataset["test"][0]["article"]
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail", device=0)
print(pipe(sample_text))

## Evaluating PEGASUS on the CNN/DailyMail Dataset

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset
import evaluate  # เปลี่ยนจาก load_metric เป็น evaluate

# 1. โหลด Dataset (ระบุชื่อโครงสร้างข้อมูล "3.0.0" เป็นพารามิเตอร์ที่สอง)
dataset = load_dataset("cnn_dailymail", "3.0.0")

# 2. โหลด Metric ผ่าน Library evaluate
rouge_metric = evaluate.load("rouge")

# 3. กำหนดชื่อ ROUGE สำหรับใช้ในตอนแสดงผล
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

print("Setup Complete!")

In [ ]:
def evaluate_summaries_baseline(dataset, metric,
                                column_text="article",
                                column_summary="highlights"):
    summaries = [three_sentence_summary(text) for text in dataset[column_text]]
    metric.add_batch(predictions=summaries,
                     references=dataset[column_summary])
    score = metric.compute()
    return score

In [ ]:
test_sampled = dataset["test"].shuffle(seed=42).select(range(1000))

# คำนวณคะแนน (ตรวจสอบให้แน่ใจว่าฟังก์ชัน evaluate_summaries_baseline ใช้ rouge_metric.compute() แล้ว)
score = evaluate_summaries_baseline(test_sampled, rouge_metric)

# ดึงคะแนนออกมาแบบตรงๆ คูณ 100
rouge_dict = {rn: score[rn] * 100 for rn in rouge_names}

# สร้าง DF แสดงผล
pd.DataFrame.from_dict(rouge_dict, orient="index", columns=["baseline"]).T

In [ ]:
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def evaluate_summaries_pegasus(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        # ใช้ inference_mode เพื่อความเร็วและประหยัดแรม
        with torch.inference_mode():
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                             attention_mask=inputs["attention_mask"].to(device),
                             length_penalty=0.8, num_beams=2, max_length=64)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # ตรงนี้สำคัญ: evaluate คืนค่าเป็น dict ของ float ทันที
    score = metric.compute()
    return score

In [ ]:
# hide_output
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_pegasus(test_sampled, rouge_metric,
                                   model, tokenizer, batch_size=2)
rouge_dict = {rn: score[rn] for rn in rouge_names}
pd.DataFrame(rouge_dict, index=["pegasus"])

In [ ]:
# hide_input
pd.DataFrame(rouge_dict, index=["pegasus"])

In [ ]:
# import matplotlib.pyplot as plt

# # สุ่มตรวจความยาวจากชุด train
# test_sample = dataset["train"].shuffle(seed=42).select(range(1000))

# d_len = [len(tokenizer.encode(s)) for s in test_sample["article"]]
# s_len = [len(tokenizer.encode(s)) for s in test_sample["highlights"]]

# fig, axes = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True)
# axes[0].hist(d_len, bins=20, color="C0", edgecolor="C0")
# axes[0].set_title("Article Token Length")
# axes[0].set_xlabel("Length")
# axes[0].set_ylabel("Count")
# axes[1].hist(s_len, bins=20, color="C0", edgecolor="C0")
# axes[1].set_title("Highlights Token Length")
# axes[1].set_xlabel("Length")
# plt.tight_layout()
# plt.show()

In [ ]:
# # เลือก Subset เพื่อประหยัดเวลา (ถ้าจะเอาทั้งหมดยกออกได้ครับ)
# train_subset = dataset["train"].shuffle(seed=42).select(range(500))
# val_subset = dataset["validation"].shuffle(seed=42).select(range(100))

# def convert_examples_to_features(example_batch):
#     # เปลี่ยน dialogue เป็น article
#     input_encodings = tokenizer(example_batch["article"], max_length=1024,
#                                 truncation=True)

#     with tokenizer.as_target_tokenizer():
#         # เปลี่ยน summary เป็น highlights
#         target_encodings = tokenizer(example_batch["highlights"], max_length=128,
#                                      truncation=True)

#     return {"input_ids": input_encodings["input_ids"],
#             "attention_mask": input_encodings["attention_mask"],
#             "labels": target_encodings["input_ids"]}

# dataset_cnn_pt = train_subset.map(convert_examples_to_features, batched=True)
# dataset_cnn_val_pt = val_subset.map(convert_examples_to_features, batched=True)

# columns = ["input_ids", "labels", "attention_mask"]
# dataset_cnn_pt.set_format(type="torch", columns=columns)
# dataset_cnn_val_pt.set_format(type="torch", columns=columns)

In [ ]:
# from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

# seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# training_args = TrainingArguments(
#     output_dir='pegasus-cnn-finetuned',
#     num_train_epochs=1,
#     warmup_steps=100,             # ลดลงให้เหมาะกับ Subset
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     weight_decay=0.01,
#     logging_steps=10,
#     push_to_hub=True,
#     eval_strategy='steps',
#     eval_steps=100,               # ประเมินผลทุกๆ 100 steps
#     save_steps=1e6,
#     gradient_accumulation_steps=16,
#     fp16=True
# )

# trainer = Trainer(model=model, args=training_args,
#                   tokenizer=tokenizer, data_collator=seq2seq_data_collator,
#                   train_dataset=dataset_cnn_pt,
#                   eval_dataset=dataset_cnn_val_pt)

In [ ]:
# # เริ่มเทรน
# trainer.train()

# # วัดผลด้วย ROUGE (ใช้ชุด test ของ cnn)
# test_subset = dataset["test"].shuffle(seed=42).select(range(100))
# score = evaluate_summaries_pegasus(
#     test_subset, rouge_metric, trainer.model, tokenizer,
#     batch_size=2, column_text="article", column_summary="highlights")

# rouge_dict = {rn: score[rn] * 100 for rn in rouge_names}
# pd.DataFrame(rouge_dict, index=["pegasus (fine-tuned-cnn)"])

In [ ]:
# from huggingface_hub import login
# login(token="")

In [ ]:
# # ทดสอบใช้งานด้วย Pipeline
# from transformers import pipeline
# gen_kwargs = {"length_penalty": 0.8, "num_beams": 8, "max_length": 128}

# # เปลี่ยน URL เป็น username ของคุณ
# pipe = pipeline("summarization", model="Posathip/pegasus-cnn-finetuned")

# sample_article = dataset["test"][0]["article"]
# print("Article:")
# print(sample_article[:500] + "...")
# print("\nModel Summary:")
# print(pipe(sample_article, **gen_kwargs)[0]["summary_text"])

In [ ]:
# from huggingface_hub import login
# login(token="")

In [ ]:
# # ส่งขึ้น Hub
# trainer.push_to_hub("Finetuned Pegasus on CNN subset by [ใส่ชื่อคุณ]")

## Training a Summarization Model

In [ ]:
# แก้บรรทัดนี้ครับ
dataset_samsum = load_dataset("knkarthick/dialogsum")

split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")
print(dataset_samsum["test"][0]["dialogue"])
print("\nSummary:")
print(dataset_samsum["test"][0]["summary"])

In [ ]:
# hide_input
print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")
print(dataset_samsum["test"][0]["dialogue"])
print("\nSummary:")
print(dataset_samsum["test"][0]["summary"])

### Evaluating PEGASUS on SAMSum

In [ ]:
pipe_out = pipe(dataset_samsum["test"][0]["dialogue"])
print("Summary:")
print(pipe_out[0]["summary_text"].replace(" .<n>", ".\n"))

In [ ]:

score = evaluate_summaries_pegasus(dataset_samsum["test"], rouge_metric, model,
                                   tokenizer, column_text="dialogue",
                                   column_summary="summary", batch_size=8)


rouge_dict = {rn: score[rn] * 100 for rn in rouge_names}

# แสดงผลเป็นตารางสวยๆ
pd.DataFrame(rouge_dict, index=["pegasus"])

In [ ]:
# hide_input
pd.DataFrame(rouge_dict, index=["pegasus"])

### Fine-Tuning PEGASUS

In [ ]:
d_len = [len(tokenizer.encode(s)) for s in dataset_samsum["train"]["dialogue"]]
s_len = [len(tokenizer.encode(s)) for s in dataset_samsum["train"]["summary"]]

fig, axes = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True)
axes[0].hist(d_len, bins=20, color="C0", edgecolor="C0")
axes[0].set_title("Dialogue Token Length")
axes[0].set_xlabel("Length")
axes[0].set_ylabel("Count")
axes[1].hist(s_len, bins=20, color="C0", edgecolor="C0")
axes[1].set_title("Summary Token Length")
axes[1].set_xlabel("Length")
plt.tight_layout()
plt.show()

In [ ]:
#hide_output
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["dialogue"], max_length=1024,
                                truncation=True)

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch["summary"], max_length=128,
                                     truncation=True)

    return {"input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]}

dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features,
                                       batched=True)
columns = ["input_ids", "labels", "attention_mask"]
dataset_samsum_pt.set_format(type="torch", columns=columns)

In [ ]:
#hide_input

#id teacher-forcing
#alt Decoder input and label alignemt for text generation.
#caption Decoder input and label alignemt for text generation.
text = ['PAD','Transformers', 'are', 'awesome', 'for', 'text', 'summarization']
rows = []
for i in range(len(text)-1):
    rows.append({'step': i+1, 'decoder_input': text[:i+1], 'label': text[i+1]})
pd.DataFrame(rows).set_index('step')

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='pegasus-samsum',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    push_to_hub=True,
    eval_strategy='steps',    
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16,
    fp16=True               
)

In [ ]:
from huggingface_hub import login
login(token="")

In [ ]:
# hide_output
trainer = Trainer(model=model, args=training_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])

In [ ]:
# hide_output

score = evaluate_summaries_pegasus(
    dataset_samsum["test"], rouge_metric, trainer.model, tokenizer,
    batch_size=2, column_text="dialogue", column_summary="summary")


# ใช้ Dictionary Comprehension เพื่อดึงค่ามาคูณ 100 ให้ดูง่ายขึ้น
rouge_dict = {rn: score[rn] * 100 for rn in rouge_names}

# 3. แสดงผลเป็นตาราง
pd.DataFrame(rouge_dict, index=["pegasus"])

In [ ]:
rouge_dict = {rn: score[rn] * 100 for rn in rouge_names}

# แสดงผลเป็นตารางสวยๆ
import pandas as pd
df = pd.DataFrame(rouge_dict, index=["pegasus (fine-tuned)"])
display(df)

In [ ]:
# hide_input
pd.DataFrame(rouge_dict, index=[f"pegasus"])

In [ ]:
# hide_output
trainer.push_to_hub("Training complete!")

### Generating Dialogue Summaries

In [ ]:
# hide
import transformers
transformers.logging.set_verbosity_error()

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}
sample_text = dataset_samsum["test"][1]["dialogue"]
reference = dataset_samsum["test"][1]["summary"]
pipe = pipeline("summarization", model="Posathip/pegasus-samsum")

print("Dialogue:")
print(sample_text)
print("\nReference Summary:")
print(reference)
print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

In [ ]:
custom_dialogue = """\
Thom: Hi guys, have you heard of transformers?
Lewis: Yes, I used them recently!
Leandro: Indeed, there is a great library by Hugging Face.
Thom: I know, I helped build it ;)
Lewis: Cool, maybe we should write a book about it. What do you think?
Leandro: Great idea, how hard can it be?!
Thom: I am in!
Lewis: Awesome, let's do it together!
"""
print(pipe(custom_dialogue, **gen_kwargs)[0]["summary_text"])

## Conclusion